In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from math import ceil
import torch
from torch.utils.data import DataLoader
import torch.optim as optim
import torch.nn as nn

import sys
sys.path.append('..')
from utils.input_pipeline import get_image_folders
from utils.training import train, optimization_step
    
torch.cuda.is_available()

In [ ]:
torch.backends.cudnn.benchmark = True

# Create data iterators

In [ ]:
batch_size = 64

In [ ]:
train_folder, val_folder = get_image_folders()

train_iterator = DataLoader(
    train_folder, batch_size=batch_size, num_workers=4,
    shuffle=True, pin_memory=True
)

val_iterator = DataLoader(
    val_folder, batch_size=256, num_workers=4,
    shuffle=False, pin_memory=True
)

# number of training samples
train_size = len(train_folder.imgs)
train_size

In [ ]:
# number of validation samples
val_size = len(val_folder.imgs)
val_size

# Model

In [ ]:
from densenet import DenseNet
model = DenseNet()
# load the model from step4
model.load_state_dict(torch.load('model_step4.pytorch_state'))

In [ ]:
# create different parameter groups
weights = [
    p for n, p in model.named_parameters()
    if 'conv' in n or 'classifier.weight' in n
]
biases = [model.classifier.bias]
bn_weights = [
    p for n, p in model.named_parameters()
    if 'norm' in n and 'weight' in n
]
bn_biases = [
    p for n, p in model.named_parameters()
    if 'norm' in n and 'bias' in n
]

In [ ]:
params = [
    {'params': weights, 'weight_decay': 1e-4},
    {'params': biases},
    {'params': bn_weights},
    {'params': bn_biases}
]
optimizer = optim.SGD(params, lr=1e-5, momentum=0.9, nesterov=True)
loss = nn.CrossEntropyLoss().cuda()
model = model.cuda()

# Train

In [ ]:
n_epochs = 5
n_batches = ceil(train_size/batch_size)

# total number of batches in the train set
n_batches

In [ ]:
%%time
def optimization_step_fn(model, loss, x_batch, y_batch):
    return optimization_step(model, loss, x_batch, y_batch, optimizer)

all_losses = train(
    model, loss, optimization_step_fn,
    train_iterator, val_iterator,
    n_epochs=n_epochs, steps_per_epoch=n_batches, n_validation_batches=40
)
# epoch logloss  accuracy    top5_accuracy time  (first value: train, second value: val)

# Save

In [ ]:
model.cpu();
torch.save(model.state_dict(), 'model_step5.pytorch_state')